In [1]:
import os
import dotenv

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import AzureCliCredential

os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
dotenv.load_dotenv()
subscription = os.getenv(f"subscription_id")
resource_group = os.getenv(f"resource_group")
workspace = os.getenv(f"workspace_name")

ml_client = MLClient(
    AzureCliCredential(), 
    subscription, 
    resource_group, 
    workspace,
)

list(ml_client.datastores.list())

[AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'azureml_globaldatasets', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/e9eedc4b-b891-4bf4-a924-06b9c6e0f02f/resourceGroups/axsa-lab-appl-fishsonar-rg/providers/Microsoft.MachineLearningServices/workspaces/axsa-lab-appl-fishsonar-ml/datastores/azureml_globaldatasets', 'Resource__source_path': '', 'base_path': '/Users/nicolaspelzmann/Documents/MTA/projects/AXH-SonarFish/run_on_azure', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x1768a0e90>, 'credentials': {'type': 'sas'}, 'container_name': 'globaldatasets', 'account_name': 'mmstoragewesteurope', 'endpoint': 'core.windows.net', 'protocol': 'https'}),
 AzureFileDatastore({'type': <DatastoreType.AZURE_FILE: 'AzureFile'>, 'name': 'workspaceworkingdirectory', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/e9eedc4b-b891-4bf4-a92

In [2]:
plant = 'lavey'
sonar_location = 'passe3'
date = "2024-03-06"
datastore_name = 'workspaceblobstore'
base_path_on_datastore = f'{plant}_videos/{sonar_location}/'
base_intermediate_path_on_datastore = f'{plant}_tracking_intermediate_data/{sonar_location}/'
base_output_path_on_datastore = f'{plant}_tracking_output_alternative_algo_settings/{sonar_location}/'
path_on_datastore = f'{base_path_on_datastore}{date.replace("-", "/")}'
intermediate_path_on_datastore = f'{base_intermediate_path_on_datastore}{date.replace("-", "/")}'
output_path_on_datastore = f'{base_output_path_on_datastore}{date.replace("-", "/")}'

uri = f'azureml://subscriptions/{subscription}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/{datastore_name}/paths/'
uri_input = f'{uri}{path_on_datastore}'
uri_intermediate_data = f'{uri}{intermediate_path_on_datastore}'
uri_output = f'{uri}{output_path_on_datastore}'
# remove sanity_check from the uri_train_val_gt_data if doing sanity check
uri_train_val_gt_data = f"{uri}{plant}_classification/train_data/{sonar_location.replace('_sanity_check', '')}/"
classification_settings_file = f"classification_settings_{plant}_{sonar_location.replace('_sanity_check', '')}.yaml"


# Run all steps in pipeline

In [3]:
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline

run_tracking = load_component(source="./components/kalman_tracking/tracking.yml")
run_classification = load_component(source="./components/classification/classification.yml")

@pipeline(
    compute="Standard-D1-v2",
)
def tracking_all_steps_incl_pre_labeling(
    input_videos_dir: Input(type=AssetTypes.URI_FOLDER), 
    train_val_gt_data_dir: Input(type=AssetTypes.URI_FOLDER),
    classification_settings_file: str,
    output_data_uri: str = None,
    intermediate_data_uri: str = None,
    log_level: str = "INFO",
):
    
    tracking_results = run_tracking(
        data=input_videos_dir,
        tracking_config="kalman_tracking_settings.yaml",
    )
    tracking_results.outputs.detections = Output(type="uri_folder", path=intermediate_data_uri, mode=InputOutputModes.RW_MOUNT)
    
    pre_labeling = run_tracking(
        data=input_videos_dir,
        labels_dir=tracking_results.outputs.detections,
        tracking_config="pre_labeling_settings.yaml",
    )
    pre_labeling.compute = "Standard-D1-v2"
    pre_labeling.outputs.detections = Output(type="uri_folder", path=intermediate_data_uri, mode=InputOutputModes.RW_MOUNT)
    
    classification_run_results = run_classification(
        classification_settings_file=classification_settings_file,
        train_val_gt_data_dir=train_val_gt_data_dir,
        files_to_classify_dir=tracking_results.outputs.detections,
        log_level=log_level,
    )
    classification_run_results.outputs.classified_detections_dir = Output(
        type="uri_folder",
        path=output_data_uri,
        mode=InputOutputModes.RW_MOUNT,
    )
    classification_run_results.compute = "Standard-A4m-v2"
    
    labeling_results = run_tracking(
        data=input_videos_dir,
        labels_dir=classification_run_results.outputs.classified_detections_dir,
        tracking_config="annotate_video_settings.yaml",
    )
    labeling_results.compute = "Standard-D1-v2"

    return {"detections": labeling_results.outputs.detections}

@pipeline(
    compute="Standard-D1-v2",
)
def tracking_all_steps(
    input_videos_dir: Input(type=AssetTypes.URI_FOLDER), 
    train_val_gt_data_dir: Input(type=AssetTypes.URI_FOLDER),
    classification_settings_file: str,
    output_data_uri: str = None,
    intermediate_data_uri: str = None,
    log_level: str = "INFO",
):
    
    tracking_results = run_tracking(
        data=input_videos_dir,
        tracking_config="kalman_tracking_settings.yaml",
    )
    tracking_results.outputs.detections = Output(type="uri_folder", path=intermediate_data_uri, mode=InputOutputModes.RW_MOUNT)
    
    classification_run_results = run_classification(
        classification_settings_file=classification_settings_file,
        train_val_gt_data_dir=train_val_gt_data_dir,
        files_to_classify_dir=tracking_results.outputs.detections,
        log_level=log_level,
    )
    classification_run_results.outputs.classified_detections_dir = Output(
        type="uri_folder",
        path=output_data_uri,
        mode=InputOutputModes.RW_MOUNT,
    )
    classification_run_results.compute = "Standard-A4m-v2"
    
    labeling_results = run_tracking(
        data=input_videos_dir,
        labels_dir=classification_run_results.outputs.classified_detections_dir,
        tracking_config="annotate_video_settings.yaml",
    )
    labeling_results.compute = "Standard-D1-v2"

    return {"detections": labeling_results.outputs.detections}

@pipeline(
    compute="Standard-D1-v2",
)
def pre_labeling(
    input_videos_dir: Input(type=AssetTypes.URI_FOLDER), 
    classified_detections_dir: Input(type=AssetTypes.URI_FOLDER),
    intermediate_data_uri: str,
):
    
    pre_labeling_results = run_tracking(
        data=input_videos_dir,
        labels_dir=classified_detections_dir,
        tracking_config="pre_labeling_settings.yaml",
    )
    pre_labeling_results.outputs.detections = Output(type="uri_folder", path=intermediate_data_uri, mode=InputOutputModes.RW_MOUNT)

    return {"pre_labeled_videos": pre_labeling_results.outputs.detections}
    

@pipeline(
    compute="Standard-D1-v2",
)
def tracking_base_steps(
    input_data: Input(type=AssetTypes.URI_FOLDER), 
    train_val_data: Input(type=AssetTypes.URI_FOLDER), 
    train_val_gt_data: Input(type=AssetTypes.URI_FOLDER),
    classification_settings_file: str,
    indermediate_data_uri: str = None,
    log_level: str = "INFO",
):
    
    tracking_results = run_tracking(
        data=input_data,
        tracking_config="kalman_tracking_settings.yaml",
    )
    tracking_results.outputs.detections = Output(type="uri_folder", path=indermediate_data_uri, mode=InputOutputModes.RW_MOUNT)
    
    classification_run_results = run_classification(
        train_val_data=train_val_data,
        train_val_gt_data=train_val_gt_data,
        files_to_classify=tracking_results.outputs.detections,
        log_level=log_level,
    )
    classification_run_results.compute = "Standard-A4m-v2"
    
    return {"detections": classification_run_results.outputs.classified_detections_dir}


@pipeline(
    compute="Standard-D1-v2",
)
def classification_and_labeling_videos(
    input_videos_dir: Input(type=AssetTypes.URI_FOLDER),
    train_val_gt_data_dir: Input(type=AssetTypes.URI_FOLDER),
    classification_settings_file: str,
    intermediate_data: Input(type=AssetTypes.URI_FOLDER),
    output_data_uri: str = None,
    log_level: str = "INFO",
):
    
    classification_run_results = run_classification(
        classification_settings_file=classification_settings_file,
        train_val_gt_data_dir=train_val_gt_data_dir,
        files_to_classify_dir=intermediate_data,
        log_level=log_level,
    )
    classification_run_results.outputs.classified_detections_dir = Output(
        type="uri_folder",
        path=output_data_uri,
        mode=InputOutputModes.RW_MOUNT,
    )
    classification_run_results.compute = "Standard-A4m-v2"
    
    labeling_results = run_tracking(
        data=input_videos_dir,
        labels_dir=classification_run_results.outputs.classified_detections_dir,
        tracking_config="annotate_video_settings.yaml",
    )
    labeling_results.compute = "Standard-D1-v2"

    return {"detections": labeling_results.outputs.detections}

@pipeline(
    compute="Standard-D1-v2",
)
def labeling_videos(
    input_data: Input(type=AssetTypes.URI_FOLDER),
    labels_dir: Input(type=AssetTypes.URI_FOLDER),
):
    labeling_results = run_tracking(
        data=input_data,
        labels_dir=labels_dir,
        tracking_config="annotate_video_settings.yaml",
    )

    return {"detections": labeling_results.outputs.detections}

In [9]:
import shutil
from azure.ai.ml import load_component

start_at = "tracking_and_pre_labeling"

if start_at == "tracking":
    pipeline_job = tracking_all_steps(
        classification_settings_file=classification_settings_file,
        input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_input, mode=InputOutputModes.RO_MOUNT),
        train_val_gt_data_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_train_val_gt_data, mode=InputOutputModes.RO_MOUNT),
        intermediate_data_uri=uri_intermediate_data,
        output_data_uri=uri_output,
    )
    pipeline_job.outputs.detections = Output(type="uri_folder", path=uri_output, mode=InputOutputModes.RW_MOUNT)
    pipeline_job.tags = {"date": date}
elif start_at == "tracking_and_pre_labeling":
    pipeline_job = tracking_all_steps_incl_pre_labeling(
        classification_settings_file=classification_settings_file,
        input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_input, mode=InputOutputModes.RO_MOUNT),
        train_val_gt_data_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_train_val_gt_data, mode=InputOutputModes.RO_MOUNT),
        intermediate_data_uri=uri_intermediate_data,
        output_data_uri=uri_output,
    )
    pipeline_job.outputs.detections = Output(type="uri_folder", path=uri_output, mode=InputOutputModes.RW_MOUNT)
    pipeline_job.tags = {"date": date}
elif start_at == "pre_labeling":
    pipeline_job = pre_labeling(
        input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_input, mode=InputOutputModes.RO_MOUNT),
        classified_detections_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_intermediate_data, mode=InputOutputModes.RO_MOUNT),
        intermediate_data_uri=uri_intermediate_data,
    )
    pipeline_job.outputs.pre_labeled_videos = Output(type="uri_folder", path=uri_intermediate_data, mode=InputOutputModes.RW_MOUNT)
    pipeline_job.tags = {"date": date}
elif start_at == "classification":
    pipeline_job = classification_and_labeling_videos(
        input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_input, mode=InputOutputModes.DOWNLOAD),
        train_val_gt_data_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_train_val_gt_data, mode=InputOutputModes.DOWNLOAD),
        indermediate_data_uri=uri_intermediate_data,
    )
    pipeline_job.outputs.classified_detection_videos_dir = Output(type="uri_folder", path=uri_output, mode=InputOutputModes.RW_MOUNT)
    pipeline_job.tags = {"date": date}
elif start_at == "labeling_videos":
    pipeline_job = labeling_videos(
        input_data=Input(type=AssetTypes.URI_FOLDER, path=uri_input, mode=InputOutputModes.DOWNLOAD),
        labels_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_output, mode=InputOutputModes.DOWNLOAD),
    )
    pipeline_job.outputs.detections = Output(type="uri_folder", path=uri_output, mode=InputOutputModes.RW_MOUNT)
    pipeline_job.tags = {"date": date}

pipeline_job.display_name = f"{plant}-{sonar_location}-{start_at}-{date}"

# copy library files to job source directory temporarily
pth_cls = './components/classification/src/analysis/classification_utils/'
pth_masks = './components/classification/src/analysis/demo/'
pth = './components/kalman_tracking/src/algorithm/'
shutil.copytree('../analysis/classification_utils/', pth_cls, dirs_exist_ok=True)
shutil.copytree('../analysis/demo/', pth_masks, dirs_exist_ok=True)
shutil.copytree('../algorithm/', pth, dirs_exist_ok=True)

pipeline_job_run = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name=f"track-and-classify-{plant}-{sonar_location}"
)

shutil.rmtree(pth)
shutil.rmtree(pth_masks)
shutil.rmtree(pth_cls)
pipeline_job_run

Experiment,Name,Type,Status,Details Page
track-and-classify-lavey-passe3_sanity_check,eager_berry_b495cltvb5,pipeline,NotStarted,Link to Azure Machine Learning studio


# Run Jobs for every day of the year

In [5]:
from typing import Optional
import pandas as pd
import shutil


def generate_paths(
    datastore_uri: str,
    day_list: list,
    date_separator: str = '/',
) -> tuple[str, str, str, str]:
    for date_str in day_list:
        date_str = date_str.replace('-', date_separator)
        path_on_datastore = f'{datastore_uri}{base_path_on_datastore}{date_str}/'
        intermediate_path_on_datastore = f'{datastore_uri}{base_intermediate_path_on_datastore}{date_str}/'
        output_path_on_datastore = f'{datastore_uri}{base_output_path_on_datastore}{date_str}/'

        yield path_on_datastore, intermediate_path_on_datastore, output_path_on_datastore, date_str


def generate_paths_for_range(
        datastore_uri: str,
        start_date: Optional[str] = None,
        end_date: Optional[str] = None,
        date_separator: str = '/',
    ) -> tuple[str, str, str, str]:
    if start_date:
        dates = pd.date_range(start=start_date, end=end_date)
    else:
        dates = ['_test']
        
    dates = [date.strftime(f'%Y{date_separator}%m{date_separator}%d') for date in dates]
    return generate_paths(datastore_uri, dates)

In [8]:
# # for a range of dates
# date_generator = generate_paths_for_range(uri, '2024-03-05', '2024-04-30')
# for a list of dates
failed_jobs = ['2024-03-08', '2024-07-01', '2024-06-30', '2024-06-29', '2024-06-26', '2024-06-22', 
 '2024-06-21', '2024-06-20', '2024-06-10', '2024-09-15', '2024-09-25', 
 '2024-09-22', '2024-09-20', '2024-08-14',
 '2024-07-24', '2024-07-19', '2024-07-18', '2024-07-17', '2024-07-16', 
 '2024-07-13', '2024-07-12', '2024-07-11', '2024-07-10']
# canceled_jobs = ['2024-03-05', '2024-03-04', '2024-03-10', '2024-03-09', '2024-03-06', '2024-04-30', 
#  '2024-04-29', '2024-04-28', '2024-04-27', '2024-04-24', '2024-04-22', '2024-04-14', 
#  '2024-04-06', '2024-04-01']
date_generator = generate_paths(uri, failed_jobs)

In [9]:
# copy library files to job source directory temporarily
pth_cls = './components/classification/src/analysis/classification_utils/'
pth_masks = './components/classification/src/analysis/demo/'
pth = './components/kalman_tracking/src/algorithm/'
shutil.copytree('../analysis/classification_utils/', pth_cls, dirs_exist_ok=True)
shutil.copytree('../analysis/demo/', pth_masks, dirs_exist_ok=True)
shutil.copytree('../algorithm/', pth, dirs_exist_ok=True)

start_at = "tracking_and_pre_labeling"

for raw_videos_dir_path, intermediate_path_on_datastore, output_path_on_datastore, date_str in date_generator:
    if start_at == "tracking":
        pipeline_job = tracking_all_steps(
            classification_settings_file=classification_settings_file,
            input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=raw_videos_dir_path, mode=InputOutputModes.RO_MOUNT),
            train_val_gt_data_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_train_val_gt_data, mode=InputOutputModes.RO_MOUNT),
            intermediate_data_uri=intermediate_path_on_datastore,
            output_data_uri=output_path_on_datastore,
        )
        pipeline_job.outputs.detections = Output(type="uri_folder", path=output_path_on_datastore, mode=InputOutputModes.RW_MOUNT)
    elif start_at == "tracking_and_pre_labeling":
        pipeline_job = tracking_all_steps_incl_pre_labeling(
            classification_settings_file=classification_settings_file,
            input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=raw_videos_dir_path, mode=InputOutputModes.RO_MOUNT),
            train_val_gt_data_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_train_val_gt_data, mode=InputOutputModes.RO_MOUNT),
            intermediate_data_uri=intermediate_path_on_datastore,
            output_data_uri=output_path_on_datastore,
        )
        pipeline_job.outputs.detections = Output(type="uri_folder", path=output_path_on_datastore, mode=InputOutputModes.RW_MOUNT)
    elif start_at == "pre_labeling":
        pipeline_job = pre_labeling(
            input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=raw_videos_dir_path, mode=InputOutputModes.RO_MOUNT),
            classified_detections_dir=Input(type=AssetTypes.URI_FOLDER, path=intermediate_path_on_datastore, mode=InputOutputModes.RO_MOUNT),
            intermediate_data_uri=intermediate_path_on_datastore,
        )
        pipeline_job.outputs.pre_labeled_videos = Output(type="uri_folder", path=intermediate_path_on_datastore, mode=InputOutputModes.RW_MOUNT)
    elif start_at == "classification":
        pipeline_job = classification_and_labeling_videos(
            classification_settings_file=classification_settings_file,
            input_videos_dir=Input(type=AssetTypes.URI_FOLDER, path=raw_videos_dir_path, mode=InputOutputModes.RO_MOUNT),
            train_val_gt_data_dir=Input(type=AssetTypes.URI_FOLDER, path=uri_train_val_gt_data, mode=InputOutputModes.RO_MOUNT),
            intermediate_data=Input(type=AssetTypes.URI_FOLDER, path=intermediate_path_on_datastore, mode=InputOutputModes.DOWNLOAD),
            output_data_uri=output_path_on_datastore,
        )
        pipeline_job.outputs.detections = Output(type="uri_folder", path=output_path_on_datastore, mode=InputOutputModes.RW_MOUNT)
    elif start_at == "labeling_videos":
        pipeline_job = labeling_videos(
            input_data=Input(type=AssetTypes.URI_FOLDER, path=raw_videos_dir_path, mode=InputOutputModes.DOWNLOAD),
            labels_dir=Input(type=AssetTypes.URI_FOLDER, path=output_path_on_datastore, mode=InputOutputModes.DOWNLOAD),
        )
        pipeline_job.outputs.detections = Output(type="uri_folder", path=output_path_on_datastore, mode=InputOutputModes.RW_MOUNT)
    pipeline_job.tags = {"date": date_str, "location": f"{plant}-{sonar_location}"}
    pipeline_job.display_name = f"{plant}-{sonar_location}-{date_str}"
    
    pipeline_job_run = ml_client.jobs.create_or_update(
        pipeline_job, 
        experiment_name=f"{plant}-{sonar_location}-{start_at}",
    )
    print(f'submitted job with tags: {pipeline_job_run.tags}')
    
shutil.rmtree(pth)
shutil.rmtree(pth_masks)
shutil.rmtree(pth_cls)

submitted job with tags: {'date': '2024/03/08', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/07/01', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/30', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/29', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/26', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/22', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/21', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/20', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/06/10', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/09/15', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/09/25', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/09/22', 'location': 'lavey-passe3'}
submitted job with tags: {'date': '2024/09/20', 'location': 'lavey-passe3'}
submitted jo